# 🔥 FHIR 101 - A Practical Guide

## Notebook Setup
If you haven't already setup your development environment please do so by following the instructions on the `kf-model-fhir` GitHub repository's [README](https://github.com/kids-first/kf-model-fhir#installation)

Next, run the cell below and then refresh this page in the browser.

In [1]:
%%capture
%%bash
pip install jupyter jupyterthemes jupyter_contrib_nbextensions
jupyter contrib nbextension install
jupyter nbextension enable collapsible_headings/main codefolding/main toc2/main
jt -t solarizedl -cursw 3 -cellw 1000 -tf sourcesans

## Hello! What is this Guide?

If any of the following statements applies to you or you've had any of the questions below, then this guide is for you!
- 🤷‍♀️I've heard the term FHIR. But is it a server? A database? A data model? Rapid oxidation?
- 🙀I understand what FHIR is but have no idea how to implement it!
- 😕I tried reading through [HL7 FHIR](https://www.hl7.org/fhir/) but it is confusing and difficult to navigate
- 🤔How does FHIR modeling relate to relational databases?   

The primary purpose of this notebook is to quickly get you educated on what FHIR is and walk you through practical exercises that will teach you the FHIR basics:
1. Create and validate a FHIR data model for an entity
2. Add/remove simple and complex attributes to/from an entity in the model
3. Make entity attributes searchable in the FHIR server
4. Deploy the FHIR data model into a FHIR server
5. Load data into the FHIR server which conforms to the model
6. Search for data in the FHIR server

**If you don't care to know background and want to get right to development, head to the Tutorial section!**

### Icons in this Guide

📘A link to a useful external reference related to the section the icon appears in

⚡️A key take away for the section the icon appears in 

## What is FHIR

Let's start with understanding exactly what FHIR is. FHIR stands for `Fast Healthcare Interoperability Resources` and was created by the organization `Health Level 7` or `HL7`.  

There is a large active international community that helps develop the specification. You can read more about it here: 

📘Learn more about [FHIR](http://hl7.org/fhir/index.html)

📘Learn more about [HL7](http://hl7.org)

### FHIR Is ...
A specification that includes all of the following:
- A loosely defined [base data model](https://www.hl7.org/fhir/resourcelist.html) describing things in healthcare (e.g. Patient, Specimen) and how they relate to     each other. The base data model also includes definitions and restrictions on server functionality
- How to extend or change the base data model to fit different healthcare use cases
- A database agnostic domain specific language (DSL) for developing files that make up the FHIR data model
- A [RESTful web API specification](https://www.hl7.org/fhir/exchange-module.html) to create, read, update, delete, and search FHIR data in a FHIR server
- A RESTful query language that describes how to construct search queries for a FHIR server
- Standards for how consumers of FHIR data should be authenticated and authorized access to health data.

### FHIR is NOT ...
- A database
- A database schema
- A server

It is important to understand that the FHIR specification is completely technology agnostic. Thus, it does not depend on programming languages or include things like relational database schemas. It is up to the implementers to decide how to implement the data model (i.e. relational database, nosql database, etc) and RESTful API. 

## Concepts

The FHIR spec is large and daunting, so we cannot cover all of the major concepts in FHIR. However, this section and the next should give you all you need to understand what you're doing in the tutorial. 

📘A good resource to learn [FHIR Basics](https://smilecdr.com/docs/tutorial_and_tour/fhir_basics.html#fhir-basics)

### FHIR Resource 

The term "Resource" can be confusing as it is sometimes used to describe different concepts in FHIR. 

This diagram should help illustrate the difference between a Resource, type of Resource, and group/category of Resource types:

![FHIR Concepts](static/images/resource-breakdown.png)

#### Resource Groups, Types, and Instances
**Resource** - an instance of a type of thing. A Resource has an identifier. Example resources are:
- Patient `PT-001`
- Specimen `SP-001`
- StructureDefinition `KidsFirst-Patient`
- SearchParameter `patient-race`
- ...

**Resource Type** - a type of a thing. Example resource types are:
- Patient
- Specimen
- StructureDefinition
- SearchParameter
- ...

**Resource Type Category/Group** - a logical grouping of Resources. Examples are:
- Conformance Resources
- Terminology Resources
- ...

The most important categories of resource types for the purposes of this tutorial are `Conformance Resources` and `Terminology Resources` because these are the ones we will create in order to build a FHIR data model. 

**A `Patient` resource might look like this:**
```json
{
    "resourceType":"Patient",
    "id": "PT-001",
    "meta": {
        "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
    },
    "gender": "male",
    "name": [
        {
            "use": "usual",
            "family":"Smith",
            "given": "Jack",
            "period": {
                "end": "2001-01-01"
            }
        }
    ]
}
``` 

📘Learn more about [FHIR Resources](https://www.hl7.org/fhir/resource.html)

### FHIR Data Types

FHIR resource models use a set of data types for holding their values. Data types can be primitive, meaning that they hold a simple value (typically a string, a number, etc.). Other data types are composite, meaning they are a structure containing several elements. Some examples are:

**Primitive Data Types**

- String - a string of text - `female`
- Decimal - an abitrary precision float - `3.14`

**Composite Data Types**

- HumanName - A person's complete name, broken into individual parts
```json
{
   "family": "Smith",
   "given": [ "John", "Frank" ],
   "suffix": [ "Jr" ]
}
```

📘Learn about all [FHIR Data Types](https://www.hl7.org/fhir/datatypes.html)

### FHIR Versions

Changes to the FHIR specification are tracked via different versions. A FHIR version has a name 
and a semantic version number. The biggest differences among versions are usually structural and content
changes in the FHIR base model definitions.

The most current and commonly used versions are:

- STU3 or version 3.0.2
- R4 or version 4.0.1

**This tutorial will always use version R4**

📘Learn more about [FHIR Versions](https://www.hl7.org/fhir/versions.html#versions)

## Model Development

What does it mean to develop a FHIR data model? Well, its not entirely different from developing any kind of data model. Developing a data model usually consists of the following steps: 

1. Define entities in the model and their attributes
2. Define relationships between entities
3. Define constraints on entity attributes (i.e. types, min/max, one of, etc)
4. Define constraints on entity relationships (i.e. Specimen must have one Patient, DiagnosticReport must have one      Patient, etc.)
5. Validate the model

More concretely, developing a data model in FHIR means:
1. Authoring JSON files containing `Conformance Resource` and `Terminology Resource` payloads
2. Validating `Conformance Resource` and `Terminology Resource` using one of the standalone HL7 validation tools or POSTing the resource payload to a FHIR server

⚡️**Key Take Away** - when we say "model" in FHIR, we are referring to `Conformance Resources` and `Terminology Resources`

### Conformance Resources

There are several different types of `Conformance Resources` but the ones that are used most in FHIR model development are: 

- `StructureDefinition` - this resource captures steps 1 through 4 above for healthcare entities like `Patient`, `Specimen`, `DiagnosticReport`, etc.

- `SearchParameter` - this resource is used to specify an attribute on an entity (Patient.gender) which can be used to search for instances of that entity on a FHIR server and what keyword/path (Patient.gender.code.value) should be used when searching.

We will take a deep dive into both of these in the tutorial.

📘Learn more about [Conformance Resources](https://www.hl7.org/fhir/conformance-module.html)

### Terminology Resources

Some entity attributes might need to be constrained to a particular value set which may come from a particular terminology system (or ontology) which defines attributes, their set of valid values, and an alphanumeric code that uniquely identifies each value.

For example, we may want to constrain the `bodySite` attribute on `Specimen` to only have values from the NCIT ontology's `Specimen Source Site` value set. We can do this by defining `Terminology Resource`s and referencing them in the `StructureDefinition` for `Specimen`.

Types of `Terminology Resource`s are:

- `CodeSystem` - describes an Ontology, Terminology, or Enumeration

- `ValueSet` - specifies a set of code and values drawn from one or more code systems

You will see a practical example of this later in the tutorial.

📘Learn about [Terminology Resources](https://www.hl7.org/fhir/terminologies.html)

### Model Documentation

#### Implementation Guide
FHIR data models are documented in a static website called an Implementation Guide. It has a standard format and looks much like the HL7 FHIR website. Implementation Guides describe and visually illustrate:

- The model's `StructureDefinition`s
- The differences between the model's `StructureDefinition`s and the ones they extend
- Example Resources for each `StructureDefinition`
- The searchable attributes on each Resource

There is also a FHIR resource, `ImplementationGuide`, which is a type of `Conformance Resource` used to store configuration information for generating the static site. 

#### Create and Publish
Implementation Guides are generated using the [HL7 IG Publisher](https://confluence.hl7.org/display/FHIR/IG+Publisher+Documentation) Java CLI. The IG Publisher also fully validates the data model during its site generation process so it is often used as a model validation tool as well. The Kids First FHIR model uses it in this way.

📘Learn more about Kids First [IG generation and model validation](https://github.com/kids-first/kf-model-fhir/blob/master/README.md) using IG Publisher 

### Model Validation

There are 2 primary ways to validate:

1. Use the [HL7 IG Publisher](https://confluence.hl7.org/display/FHIR/IG+Publisher+Documentation) Java CLI
2. Use a FHIR server
    - POST StructureDefinitions to `/StructureDefinition` endpoint
    - POST example Resources to `<resourceType>/$validate` endpoint

The most thorough and standard way to validate a FHIR data model is to use the IG Publisher. The IG Publisher is under active development and is the most up to date in terms of its validation logic. However, since the IG Publisher tightly couples IG site generation with validation it can be complicated to use. 

The second method is easier but not standard. All compliant FHIR servers will implement basic validation logic, however, some servers may or may not implement additional validation/checks that the IG Publisher does.

To keep things simple, we will use the second method of validation.

⚡️**Key Take Away** - the most thorough way to validate a FHIR data model is using the HL7 IG Publisher

## Server Development

### Pick an Existing Solution
You might have noticed that there are not many open source or free FHIR servers currently in development. This is likely due to a couple of factors:

- The FHIR specification is rapidly evolving and the tooling around it is still in early stages of development. 
- Server development is hard 

In order to develop a compliant FHIR server you must: 

- Implement the FHIR model validation logic. This is not easy since a FHIR model is authored  
in the FHIR DSL. The server must know how to parse the DSL (a.k.a conformance resources) and
execute rules defined in them.
- Implement a parser and query constructor for the FHIR RESTful query language - which is also custom/not based on any known standard. Additionally, the kinds of search queries that can be made with a FHIR search API are fairly rich and complex.

### FHIR Server Solutions
For these reasons it is best to select one of the existing FHIR server solutions rather than develop your own. From personal experience and a fair bit of research, the most promising ones are:

<table style="bgcolor: none;">
    <tbody>
        <tr>
            <td style="border-right: 1px solid #cccccc; padding: 7px;">
                <a href="https://health-samurai.webflow.io/aidbox">
                    <img src="static/images/aidbox-logo.png" alt="Health Samurai Aidbox"/>
                </a>
            </td>
            <td style="padding: 7px;">
                <a href="https://smilecdr.com">
                    <img src="static/images/smilecdr-logo.png" alt="Smile CDR"/>
                </a>
            </td>
        </tr>
    </tbody>
</table>





## Tutorial

### Real World Use Case
In this tutorial we will create a simple data model that is able to capture basic demographic information about Leukemia patients that have had a biosample drawn from them.  No protected health information [(PHI)](https://www.hipaajournal.com/considered-phi-hipaa/) data should be captured. In fact there should be no PHI attributes available anywhere in the model.

At a minimum we want to capture:

**Patient Attributes**
- Gender (required)
- Race (required)

**Specimen Attributes**
- The patient from whom the sample was drawn
- Anatomical site or location on the body from which the sample was drawn
- Composition - the type of sample - blood, tissue, saliva, etc.

### What You'll Learn
You will learn the following practical concepts in this tutorial:

- How to create a new model for a FHIR resource by extending it's base model
- How to add/remove attributes from the base model
- How to add make attributes required
- How to make an attribute searchable
- How to add a coded attribute - or one that is restricted to an enumeration defined by an ontology
- How to validate your model
- How to document your model

### Setup

#### Requirements
**FHIR Server**

- We will use [HAPI](http://hapi.fhir.org/), the reference FHIR server, for this tutorial. 

**Base URL**

- All HTTP requests will be sent to HAPI's base URL: http://10.10.1.191:8000

**Model Development Editor**

You will need something to author JSON formatted model files. Unfortunately there are almost no user friendly tools for this. Most model development is done in a code editor. This can be difficult as it requires the developer to unecessarily remember complex syntax of the FHIR DSL in order to create and edit model files.

- **Forge** - is a [visual editor](https://simplifier.net/forge) for FHIR model development.     It does make some things easier, but it can be buggy and was natively developed for windows.   It is possible to [run it on OSX](http://docs.simplifier.net/forge/forgeInstall.html#running-on-macos). You can use it, but       explanation of how to use it is out   of the scope of this tutorial.

#### Development Environment
Here we're going to import the code we need and complete the necessary setup which will be used later in the tutorial.

You don't need to do this if you would like to use another HTTP client like `Postman` to interact with the FHIR server.

In [2]:
# Be sure to execute this cell so that we can use the client later
from pprint import pprint
from kf_model_fhir.client import FhirApiClient

BASE_URL = 'http://10.10.1.191:8000'
FHIR_VERSION = '4.0.0'
client = FhirApiClient(base_url=BASE_URL, fhir_version=FHIR_VERSION)

### Create a Patient Model

The FHIR base model was intended to include many common healthcare entities
like patients, biospecimens, etc. which cover 80% of healthcare data capture
uses cases.

It is rare to come across a healthcare entity that isn't represented in
one of the existing FHIR resources. The remaining 20% of data capture use cases
that are not satisfied by the base model can easily be satisfied by extending
the base model.

So we can start by looking at the base model's `Patient` `StructureDefinition` to
see if anything needs to be modified.

The HL7 FHIR website has a nice visual representation of the `Patient`
`StructureDefinition`:

[![Patient StructureDefinition](static/images/patient-sd-visual.png)](https://www.hl7.org/fhir/patient.html#resource)
📘Learn about the [symbol definitions in the view ^](http://hl7.org/fhir/StructureDefinition/contactpoint-area)

We can already see that the `Patient` `StructureDefinition` has one attribute we want:

- `gender`

And many attributes that we'll want to **remove** because they represent PHI fields:
- `name`
- `telecom`
- `address`
- `birthDate`
- `contactPoint`
- `photo`

### Start with a Template
We've seen that StructureDefinitions are complex looking structures. It is difficult to remember what attributes are required and the exact structure. One way to create StructureDefinitions is to use an application like Forge which hides that complexity since you're working with a UI. 

However, if you don't have or want to use Forge, you can browse existing StructureDefinitions and pick one similar to what you need and use it as a template.  

- 📘Implementation Guides, published on [HL7 FHIR IG registry](https://registry.fhir.org/guides) also have links to the pack of StructureDefinitions that are documented in the IG

- 📘 StructureDefinitions are also published on [Simplifier.net](https://simplifier.net)

#### Create Initial Patient StructureDefinition

To create a custom data model for `Patient`, we must create a `StructureDefinition` which 
will extend the FHIR R4 `StructureDefinition` for `Patient`:

In [3]:
# Be sure to execute this cell so that we can use `patient_sd` later
patient_sd = {
    "resourceType": "StructureDefinition",
    "url": "http://fhir.kids-first.io/StructureDefinition/Patient",
    "version": "0.1.0",
    "name": "kids_first_research_participant",
    "title": "Kids First Research Participant",
    "status": "draft",
    "publisher": "Kids First DRC",
    "description": "The individual human or other organism.",
    "fhirVersion": "4.0.0",
    "kind": "resource",
    "abstract": False,
    "type": "Patient",
    "baseDefinition": "http://hl7.org/fhir/StructureDefinition/Patient",
    "derivation": "constraint",
    "differential": {
        "element": [
            {
                "id": "Patient",
                "path": "Patient"
            },
            {
                "id": "Patient.identifier",
                "path": "Patient.identifier",
                "mustSupport": True
            },
            {
                "id": "Patient.name",
                "path": "Patient.name",
                "max": "0"
            },
            {
                "id": "Patient.telecom",
                "path": "Patient.telecom",
                "max": "0"
            },
            {
                "id": "Patient.address",
                "path": "Patient.address",
                "max": "0"
            },
            {
                "id": "Patient.photo",
                "path": "Patient.photo",
                "max": "0"
            },
            {
                "id": "Patient.contact",
                "path": "Patient.contact",
                "max": "0"
            },
            {
                "id": "Patient.gender",
                "path": "Patient.gender",
                "min": 1
            }
        ]
    }
}

#### Anatomy of StructureDefinition

We won't cover this in great detail but here are the important parts of the `StructureDefinition`:

- `resourceType` - every FHIR resource has to declare what type it is
- `baseDefinition` - this points to the `StructureDefinition` that the current one extends. It doesn't have to be one of the base FHIR definitions. It could be a `StructureDefinition`   from any FHIR data model. This is one of the features of FHIR that helps achieve interoperability.
- `url` - this is a required unique identifier for a `Conformance Resource`. It is a stable [canonical identifier](http://hl7.org/fhir/resource.html#canonical) for the resource and is never intended to change regardless of which FHIR server uses it.
- `fhirVersion` - the FHIR semantic version number is required. It will instruct  anything validating this resource to use R4 (4.0.0) validation logic.
- `differential` or `snapshot` - Every `StructureDefinition` must have either a `differential` - a section of content that specifies differences from the base definition, or a `snapshot` section which specifies the full definition. For model development, you almost always use the `differential` section and not the `snapshot`.

📘Learn more about [StructureDefinition](http://hl7.org/fhir/structuredefinition.html) anatomy

📘Learn more about [Resource identifiers](http://hl7.org/fhir/resource.html#identification)

📘Browse [FHIR data models](https://simplifier.net/search?searchtype=Projects) from other projects

### Remove Attributes

We're going to start with the easiest modification first: removing an attribute from the base model. In fact, in our initial `Patient` `StructureDefinition`, we've already removed a handful of attributes. 

To remove an attribute you simply set its maximum cardinality to 0 in the `StructureDefinition` `differential` section: 

```json
{
    "id": "Patient.name",
    "path": "Patient.name",
    "max": "0"
}
```

### Make an Attribute Required

This is just as simple as removing an attribute. To make an attribute required you simply set its minimum cardinality to 1 in the `StructureDefinition` `differential` section::

```json
{
    "id": "Patient.gender",
    "path": "Patient.gender",
    "min": 1
}
```

### Validate the Patient Model

Validating the Kids First Patient model means 2 things:

1. Validate the Kids First `Patient` `StructureDefinition` - structure, syntax, cardinality, etc.
2. Test both valid and invalid `Patient` Resources against the Kids First Patient `StructureDefinition`

📘Learn more about [Resource Validation](https://www.hl7.org/fhir/validation.html)

#### Validate StructureDefinition

In [4]:
def validate(payload, expected_success=True, print_result=False):
    rt = payload.get('resourceType')
    endpoint = f'{client.base_url}/{rt}'

    # Let's get into the habit of deleting existing instances before we create one
    # The easiest way to do this is to search for StructureDefinitions by canonical URL and
    # then delete what is returned (should only be 1 since canonical URL is a unique ID)
    success = client.delete_all(endpoint, params={'url': payload['url']})
    print(f'DELETE {payload["url"]}')
    assert success

    # POST the Patient StructureDefinition /StructureDefinition endpoint
    success, result = client.send_request('POST', endpoint, json=payload)
    if print_result:
        pprint(result)
    print(f'POST {endpoint}, {payload.get("name")}')
    assert success == expected_success      

validate(patient_sd, print_result=True)

DELETE http://fhir.kids-first.io/StructureDefinition/Patient
{'request_url': 'http://10.10.1.191:8000/StructureDefinition',
 'response': {'abstract': False,
              'baseDefinition': 'http://hl7.org/fhir/StructureDefinition/Patient',
              'derivation': 'constraint',
              'description': 'The individual human or other organism.',
              'differential': {'element': [{'id': 'Patient', 'path': 'Patient'},
                                           {'id': 'Patient.identifier',
                                            'mustSupport': True,
                                            'path': 'Patient.identifier'},
                                           {'id': 'Patient.name',
                                            'max': '0',
                                            'path': 'Patient.name'},
                                           {'id': 'Patient.telecom',
                                            'max': '0',
                                     

#### Create a Patient Resource

In [5]:
# Let's start by creating a valid (has all required attributes) Patient resource
patient_rs = {
    "resourceType":"Patient",
    "id": "kidsfirst-test-patient",
    "meta": {
        "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
    },
    "gender": "male"
}

#### Validate the Patient Resource

To test the `Patient` resource against the Kids First Patient model you must declare that this resource conforms to the `Patient` `StructureDefinition` like this:

```json
"meta": {
    "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
}
```

In [6]:
def validate_rs(resource_type, payload, expected_success=True, 
                create_on_success=True, print_result=False):
    # Validate by POSTing to the /Patient/$validate endpoint
    endpoint = f"{client.base_url}/{resource_type}/$validate"

    # Since we know this is a valid Patient resource, it should succeed
    success, result = client.send_request('POST', endpoint, json=payload)
    if print_result:
        pprint(result)
    print(f'Validate {endpoint}, {payload.get("id")}')
    assert success == expected_success
          
    # Now actually create the Patient resource
    if create_on_success and success:
        endpoint = f"{client.base_url}/{resource_type}/{payload['id']}"
        success, result = client.send_request('PUT', endpoint, json=payload)
        print(f'PUT {endpoint}')
        assert success == expected_success  

validate_rs('Patient', patient_rs, print_result=True)

{'request_url': 'http://10.10.1.191:8000/Patient/$validate',
 'response': {'issue': [{'code': 'processing',
                         'diagnostics': 'dom-6: A resource should have '
                                        'narrative for robust management '
                                        '[text.div.exists()]',
                         'location': ['Patient', 'Line 1, Col 2'],
                         'severity': 'warning'},
                        {'code': 'processing',
                         'diagnostics': 'dom-6: A resource should have '
                                        'narrative for robust management '
                                        '[text.div.exists()]',
                         'location': ['Patient', 'Line 1, Col 2'],
                         'severity': 'warning'}],
              'resourceType': 'OperationOutcome'},
 'status_code': 200}
Validate http://10.10.1.191:8000/Patient/$validate, kidsfirst-test-patient
PUT http://10.10.1.191:8000/Patient/kidsfir

#### Validate an Invalid Patient Resource

In [7]:
# Let's create an invalid resource and make sure it fails 
# Invalid since its missing the REQUIRED gender attribute
patient_rs_invalid = {
    "resourceType":"Patient",
    "id": "kidsfirst-test-patient",
    "meta": {
        "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
    }
}
validate_rs('Patient', patient_rs_invalid, expected_success=False, print_result=True)

{'request_url': 'http://10.10.1.191:8000/Patient/$validate',
 'response': {'issue': [{'code': 'processing',
                         'diagnostics': 'dom-6: A resource should have '
                                        'narrative for robust management '
                                        '[text.div.exists()]',
                         'location': ['Patient', 'Line 1, Col 2'],
                         'severity': 'warning'},
                        {'code': 'processing',
                         'diagnostics': 'Profile '
                                        'http://fhir.kids-first.io/StructureDefinition/Patient, '
                                        "Element 'Patient.gender': minimum "
                                        'required = 1, but only found 0',
                         'location': ['Patient', 'Line 1, Col 2'],
                         'severity': 'error'}],
              'resourceType': 'OperationOutcome'},
 'status_code': 422}
Validate http://10.10.1.191:800

### Add an Attribute

#### Extensions
In FHIR, when you want to add a new attribute to a `StructureDefinition`, its known as adding an "extension" since you are extending the definition of the base model.

To add an attribute to the `Patient` `StructureDefinition`, you must do 2 things: 
1. Define an extension for the attribute by creating a `StructureDefinition` resource for it
2. Declare/reference the extension in the `Patient` `StructureDefinition` 

📘Learn more about defining [extensions](https://www.hl7.org/fhir/extensibility-examples.html)

#### Why so complicated? ----> Reusability
You might be wondering why adding a new attribute isn't as simple as removing an attribute. There are a couple of reasons for this. The primary reason is because the FHIR spec pushes for interoperability through sharing of model components like extensions.

Since extensions are defined in their own files, they can be reused in a plug and play fashion by other models. We will see this later in the tutorial when we reuse the `karyotypic-sex`extension defined by the [Phenopackets FHIR model](https://aehrc.github.io/fhir-phenopackets-ig/).

Unfortunately, if you don't care about making your attributes shareable and/or you only care about simple key-value type attributes, FHIR does not have a simpler way to define these right now.


#### Simple and Complex Extensions
Another reason why extensions may be complex is because they can represent both simple attributes or complex attributes. A simple attribute might be a key, value pair with a primitive data type and a complex attribute might be an entirely new structure that might have several other extensions in a nested hierarchy.

⚡️**Key Take Away - Reuse existing extensions as much as possible! This is a major benefit of FHIR**

### Anatomy of an Extension
Similar to StructureDefinitions, the best way to create an extension is to start with an existing extension that is similar to the one you want to create. The best place to choose from is 📘the official [HL7 FHIR extension registry](https://registry.fhir.org/)

Let's take a look at the extension we'll be adding to the Patient model to represent the patient's biological race. 

In [8]:
# Here's what a simple key-value extension with a string data type looks like:
patient_race_ext = {
  "resourceType" : "StructureDefinition",
  "extension" : [{
    "url" : "http://hl7.org/fhir/StructureDefinition/structuredefinition-wg",
    "valueCode" : "oo"
  },
  {
    "url" : "http://hl7.org/fhir/StructureDefinition/structuredefinition-fmm",
    "valueInteger" : 1
  }],
  "url" : "http://fhir.kids-first.io/StructureDefinition/patient-race",
  "version" : "4.0.0",
  "name" : "patient_race",
  "status" : "draft",
  "date" : "2020-01-13",
  "publisher" : "Kids First DRC",
  "description" : "Race of patient",
  "fhirVersion" : "4.0.0",
  "kind" : "complex-type",
  "abstract" : False,
  "context" : [{
    "type" : "element",
    "expression" : "Patient"
  }],
  "type" : "Extension",
  "baseDefinition" : "http://hl7.org/fhir/StructureDefinition/Extension",
  "derivation" : "constraint",
  "differential" : {
    "element" : [{
      "id" : "Extension",
      "path" : "Extension",
      "short" : "Race",
      "definition" : "Race of patient",
      "min" : 0,
      "max" : "1",
    },
    {
      "id" : "Extension.extension",
      "path" : "Extension.extension",
      "max" : "0"
    },
    {
      "id" : "Extension.url",
      "path" : "Extension.url",
      "fixedUri" : "http://fhir.kids-first.io/StructureDefinition/patient-race"
    },
    {
      "id" : "Extension.value[x]",
      "path" : "Extension.value[x]",
      "min" : 1,
      "type" : [{
        "code" : "string"
      }]
    }]
  }
}

Similar to our Patient model, an extension is just another `StructureDefinition` except with `type=Extension` instead of `type=Patient`. 

For all `Extension` type `StructureDefinition`:

- `type` must be set to "Extension"
- `kind` must be set to "complex-type" and not "resource" since this is an extension
- `baseDefinition` must be set to the URL of an existing valid Extension type StructureDefinition. 

Additionally, you must specify in the `differential`:

- the `url` or identifier of the extension

And for simple extensions representing a key and value with a primitive data type, you must specify in the `differential`: 

- the `value[x]` or data type of the extension's value

Later on in the tutorial you will learn how to change this from a string type extension to a more complex extension with a `CodeableConcept` data type.

### Add Race to Patient Model
So far we've just defined the new attribute we want to add to the Patient model. Now we must actually bind it or declare it in the Patient model. Here's how we do this:

In [9]:
patient_sd = {
    "resourceType": "StructureDefinition",
    "url": "http://fhir.kids-first.io/StructureDefinition/Patient",
    "version": "0.1.0",
    "name": "kids_first_research_participant",
    "title": "Kids First Research Participant",
    "status": "draft",
    "publisher": "Kids First DRC",
    "description": "The individual human or other organism.",
    "fhirVersion": "4.0.0",
    "kind": "resource",
    "abstract": False,
    "type": "Patient",
    "baseDefinition": "http://hl7.org/fhir/StructureDefinition/Patient",
    "derivation": "constraint",
    "differential": {
        "element": [
            {
                "id": "Patient",
                "path": "Patient"
            },
            {
                "id": "Patient.identifier",
                "path": "Patient.identifier",
                "mustSupport": True
            },
            {
                "id": "Patient.name",
                "path": "Patient.name",
                "max": "0"
            },
            {
                "id": "Patient.telecom",
                "path": "Patient.telecom",
                "max": "0"
            },
            {
                "id": "Patient.address",
                "path": "Patient.address",
                "max": "0"
            },
            {
                "id": "Patient.photo",
                "path": "Patient.photo",
                "max": "0"
            },
            {
                "id": "Patient.contact",
                "path": "Patient.contact",
                "max": "0"
            },
            {
                "id": "Patient.gender",
                "path": "Patient.gender",
                "mustSupport": True,
                "min": 1
            },
            {
                "id": "Patient.extension",
                "path": "Patient.extension",
                "slicing": {
                    "discriminator": [
                        {
                            "type": "value",
                            "path": "url"
                        }
                    ],
                    "ordered": False,
                    "rules": "open"
                }
            },
            {
                "id": "Patient.extension:patient-race",
                "path": "Patient.extension",
                "sliceName": "patient-race",
                "short": "Race",
                "definition": "Race of patient",
                "min": 0,
                "max": "1",
                "type": [
                    {
                        "code": "Extension",
                        "profile": [
                            "http://fhir.kids-first.io/StructureDefinition/patient-race"
                        ]
                    }
                ],
                "isModifier": False
            }
        ]
    }
}

#### Explanation of Parts
Let's break down each piece. First we'll start with the piece that says we want to use extensions in the `Patient` `StructureDefinition`:
```json
{
    "id": "Patient.extension",
    "path": "Patient.extension",
    "slicing": {
        "discriminator": [
            {
                "type": "value",
                "path": "url"
            }
        ],
        "ordered": False,
        "rules": "open"
    }
}
```
- `id`: TODO
- `path`: All extensions go inside an `extension` element in the Patient resource
- `slicing`: The `Patient.extension` is a list of extensions and each one must be able to be uniquely identified in that list. The extension's URL is used to "slice" or identify the extension in the list.
- `ordered=False`: order of extensions does not matter in a Patient resource
- `rules=open`: additional extensions can be added in a Patient resource

The next part is the actual binding of the race extension to the `Patient` `StructureDefinition`: 

```json
{
    "id": "Patient.extension",
    "path": "Patient.extension",
    "sliceName": "patient_race",
    "short": "Race",
    "definition": "Race of patient",
    "min": 0,
    "max": "1",
    "type": [
        {
            "code": "Extension",
            "profile": [
                "http://fhir.kids-first.io/StructureDefinition/patient-race"
            ]
        }
    ],
    "isModifier": False
}
```

The important parts to note here are: 

- `min/max`: this says Patient instances can only have one occurrence of the `race` extension and it is not required
- `type`: this binds the Patient's race attribute to it's `StructureDefinition`


### Validate Patient Model with Race

Now let's validate our updated Patient model and then test it against some example Patient resources which make use of the new race extension

In [10]:
# Validate/Create the Patient race extension
validate(patient_race_ext, expected_success=True, print_result=True)

# Validate/Create the updated Patient StructureDefinition which uses Race extension
validate(patient_sd, expected_success=True, print_result=True)

DELETE http://fhir.kids-first.io/StructureDefinition/patient-race
{'request_url': 'http://10.10.1.191:8000/StructureDefinition',
 'response': {'abstract': False,
              'baseDefinition': 'http://hl7.org/fhir/StructureDefinition/Extension',
              'context': [{'expression': 'Patient', 'type': 'element'}],
              'date': '2020-01-13',
              'derivation': 'constraint',
              'description': 'Race of patient',
              'differential': {'element': [{'definition': 'Race of patient',
                                            'id': 'Extension',
                                            'max': '1',
                                            'min': 0,
                                            'path': 'Extension',
                                            'short': 'Race'},
                                           {'id': 'Extension.extension',
                                            'max': '0',
                                            'pat

### Create and Validate a Patient With Race

In [11]:
patient_rs['extension'] = [
    {
        'url': 'http://fhir.kids-first.io/StructureDefinition/patient-race',
        'valueString': "Asian"
    }
]
validate_rs('Patient', patient_rs, expected_success=True)

Validate http://10.10.1.191:8000/Patient/$validate, kidsfirst-test-patient
PUT http://10.10.1.191:8000/Patient/kidsfirst-test-patient


### Search for Resources

At this point we're going to take a look at how search works in FHIR so that we can start searching for our resources using the attributes we're adding to our models. Also, this will come in handy for debugging - to find and verify that we've created resources we intended to. 

#### FHIR Search DSL
To search for FHIR resources on a FHIR server you need to be able to construct search queries in some sort of language the server understands and then send those queries to the server. 

The FHIR Search specification defines its own RESTful query language to accomplish this. This means that search queries are constructed using the URL and its query parameters of the HTTP request. 

Why does FHIR define another DSL for this? Because there isn't really a widely accepted standard for a RESTful query language. Operators like `and`, `or`, `>`, `>=`, etc can all be represented in different ways. 

#### FHIR Search Queries
FHIR search queries look like this:

```
# Format
GET [base]/[type]?name=value&...

# Example
GET http://10.10.1.191:8000/Patient?gender=female&name=Natasha
``` 

📘Learn more about the [FHIR Search Spec](https://www.hl7.org/fhir/search.html)

📘Check out this [Jupyter Notebook]() for a demo of FHIR search features ()

#### SearchParameter Resource
So you know what search queries look like, but how do you know what attributes of a Resource or keywords you can use in search queries? All attributes? Some of them? Are keywords the same as attributes?

The `SearchParameter` is a type of `Conformance Resource` which answers this question. 

In order to search for a Resource by a particular keyword or Resource attribute, the FHIR server must have a `SearchParameter` defined for it. 

Each of the FHIR Resources in the FHIR base model, have a set of predefined SearchParameters that make a subset of the Resources' attributes searchable - by either the name of the attribute or some other keyword used to alias it.

#### Patient Built-In Search Parameters
You can always check the Resource type page on the HL7 FHIR site to see which search parameters are already supported for Resources of that type. For example here is the page for `Patient`

![Patient Search Params](static/images/patient-search-params.png)


#### Name vs Expression

Something very important to note about `SearchParameter` is the differentiation between it's name and expression. 

Take a look at the `family` `SearchParameter`. A `Patient` resource doesn't have a first-level attribute called `family`. In fact, `family` is nested inside the `name` list: 

```
{
    "resourceType":"Patient",
    "id": "test-patient",
    "gender": "male",
    "name": [
        {
            "use": "usual",
            "family":"Smith",
            "given": "Jack",
            "period": {
                "end": "2001-01-01"
            }
        }
    ]
}
```

However, the expression says that when you search for Patients by `family=Smith` it will actually search for all Patients with a name element in the `name` list where `family` equals `Smith`.

The search query for this would look like this:
```
GET http://10.10.1.191:8000/Patient?family=Smith
```

This is very powerful because you can:

- Search for Resources using a keyword which finds instances by some abitrary attribute in the Resource payload
- Turn on/off search functionality by including/excluding SearchParameters in the server
- Reuse a SearchParameter by making it apply to multiple Resource Types
- Share search functionality across servers by sharing the SearchParameter definition file

**⚡️Key Take Away:** SearchParameter allows you to specify a keyword that can be used to search for Resources by some arbitrary attribute on that Resource even if its nested

#### SearchParameter Types

There is a lot of information on this topic so we cannot cover all of it. However, it is important to understand that SearchParameters have different [types](https://www.hl7.org/fhir/search.html#ptypes) which govern which modifiers can be used as a prefix/suffix to either the name or value of the search parameter in a search query. 

For example, for `number`, `date`, and `quantity` type SearchParameters, we can use one of the available modifiers like `gt` in a query:

```
GET http://10.10.1.191:8000/DiagnosticReport?date=gt2013-01-14
```

This query will return us all `DiagnosticReport` resources that occurred after January 14, 2013.


### Search for Patients by Race

Ok, now let's create a SearchParameter for the `race` attribute in our Patient model. This is going to consist of 2 steps:

1. Create a SearchParameter for Patient.race
2. Load the SearchParameter into the server

#### Create Search Parameter for Patient.race

In [15]:
patient_race_sp = {
  "resourceType": "SearchParameter",
  "url": "http://fhir.kids-first.io/SearchParameter/patient-race",
  "name": "patient-race",
  "status": "active",
  "experimental": True,
  "publisher": "Kids First DRC",
  "description": "Search for Patients by their race",
  "code": "race",
  "base": [
    "Patient"
  ],
  "type": "string",
  "expression": "Patient.extension.where(url = 'http://fhir.kids-first.io/StructureDefinition/patient-race').value",
  "target": [
    "Patient"
  ]
}

### Anatomy of SearchParameter

Again, we won't go over everything, but let's take a look at some of the important parts of a `SearchParameter`:

- `code`: this is the keyword that will be used in search queries
- `name`: a computer friendly name for the search parameter
- `base`: list of Resource types for which this SearchParameter applies
- `target`: this seems completely redundant with `base`. Not sure why it exists, but if you don't include your searches won't work
- `type`: must be one of the valid search parameter types
- `expression`: the path to the attribute value in the resource payload which is being used in the search query. This is specified as a `FHIRpath` expression which is based on [xpath](https://www.w3schools.com/xml/xpath_intro.asp)

📘Learn more about [FHIRpath expressions](https://www.hl7.org/fhir/fhirpath.html)

### Validate Patient Race Search Parameter

In [16]:
validate(patient_race_sp, print_result=True)

DELETE http://fhir.kids-first.io/SearchParameter/patient-race
{'request_url': 'http://10.10.1.191:8000/SearchParameter',
 'response': {'base': ['Patient'],
              'code': 'race',
              'description': 'Search for Patients by their race',
              'experimental': True,
              'expression': 'Patient.extension.where(url = '
                            "'http://fhir.kids-first.io/StructureDefinition/patient-race').value",
              'id': '4842',
              'meta': {'lastUpdated': '2020-01-16T15:58:18.282+00:00',
                       'versionId': '1'},
              'name': 'patient-race',
              'publisher': 'Kids First DRC',
              'resourceType': 'SearchParameter',
              'status': 'active',
              'target': ['Patient'],
              'type': 'string',
              'url': 'http://fhir.kids-first.io/SearchParameter/patient-race'},
 'status_code': 201}
POST http://10.10.1.191:8000/SearchParameter, patient-race


### Search for Patient by Race

In [17]:
def search(resource_type, params, print_result=False):
    endpoint = f'{client.base_url}/{resource_type}'
    success, result = client.send_request('GET', endpoint, params=params)
    print(f'Searching for {result["request_url"]}, '
          f'total found: {result["response"].get("total")}')
    if print_result:
          pprint(result)      
    assert success      
search('Patient', {'race': 'Asian'}, print_result=True)

Searching for http://10.10.1.191:8000/Patient?race=Asian, total found: 0
{'request_url': 'http://10.10.1.191:8000/Patient?race=Asian',
 'response': {'id': '12b7ec1f-b8cc-400e-8c3f-973bfc5b4740',
              'link': [{'relation': 'self',
                        'url': 'http://localhost:8000/Patient?race=Asian'}],
              'meta': {'lastUpdated': '2020-01-16T15:57:23.852+00:00'},
              'resourceType': 'Bundle',
              'total': 0,
              'type': 'searchset'},
 'status_code': 200}
